In [1]:
from sunrgbd import SunRGBD_Dataset
import cv2
import numpy as np
from cameramodels import PinholeCameraModel
import open3d as o3d
from PIL import Image

DATASET_PATH = "SUNRGBD/kv1/NYUdata/"
data = SunRGBD_Dataset(DATASET_PATH)

In [109]:
def get_pointcloud(depth_image, intrinsics, extrinsics = None, R = np.eye(3)):
    camera = PinholeCameraModel(depth_image.shape[0], depth_image.shape[1], intrinsics, extrinsics, R = R)
    depth_image = camera.rectify_image(depth_image, interpolation=Image.NEAREST)
    pc = camera.depth_to_points(depth_image)
    pc = pc.reshape(-1, 3)
    idx = np.argwhere(pc == np.array([0, 0, 0]))
    pc = np.delete(pc, idx, axis=0) / 10000
    return pc

In [2]:
rec = data.records[200]
n = rec.get_number_of_objects()
ann_2d = rec.get_segments_2d()
depth_image = cv2.imread(rec.depth_path, cv2.IMREAD_UNCHANGED)
rgb_image = cv2.imread(rec.image_path)
K, Rt = rec.intrinsics.reshape(3, 3), rec.extrinsics
estrinsics = np.eye(4)
estrinsics[:3, :] = Rt
P = np.dot(K, Rt)
R = Rt[:, :3]

In [10]:
estrinsics

array([[ 0.999677, -0.023916, -0.006473,  0.      ],
       [ 0.024777,  0.964971,  0.261185,  0.      ],
       [-0.      , -0.261261,  0.965251,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  1.      ]])

In [11]:
K

array([[518.857901,   0.      , 284.582449],
       [  0.      , 519.469611, 208.736166],
       [  0.      ,   0.      ,   1.      ]])

In [4]:
camera = o3d.camera.PinholeCameraIntrinsic(depth_image.shape[1], depth_image.shape[0], K)
depth_image = cv2.imread(rec.depth_path, cv2.IMREAD_UNCHANGED)
depth_image = o3d.geometry.Image(depth_image)

In [5]:
pc = o3d.geometry.PointCloud.create_from_depth_image(depth_image, camera, estrinsics, depth_scale=10000.0, depth_trunc=10000.0)

In [145]:
room = o3d.utility.Vector3dVector(rec.get_room_polygon())
print(room)
room_obb = o3d.geometry.OrientedBoundingBox.create_from_points(room)
room_obb.color = (1, 0, 0)

std::vector<Eigen::Vector3d> with 8 elements.
Use numpy.asarray() to access data.


In [7]:
def project_box(box, K, estrinsics):
    homo_box = np.hstack((box, np.ones((box.shape[0], 1))))
    box_world_frame = homo_box @ estrinsics.T
    return box_world_frame[:, :3]


In [12]:
obbs = []
for oi in range(n):
    ot = rec.get_object_type_by_index(oi)
    if ot is not None and rec.is_polygon_a_box(oi):
        obb = np.array(rec.get_object_polygon_by_index(oi))
        print(obb)
        #obb = project_box(obb, K, estrinsics)
        OBB = o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(obb))
        OBB.color = (0, 1, 0)
        obbs.append(OBB)

[[ 0.11212933  0.79943195  3.7250849 ]
 [ 0.11212933  0.34943192  3.7250849 ]
 [-0.01916831  0.79943195  3.15670871]
 [-0.01916831  0.34943192  3.15670871]
 [-0.46697985  0.79943195  3.2601553 ]
 [-0.46697985  0.34943192  3.2601553 ]
 [-0.33568222  0.79943195  3.82853149]
 [-0.33568222  0.34943192  3.82853149]]
[[0.41029568 0.79943195 3.95556033]
 [0.41029568 0.37443192 3.95556033]
 [1.00916657 0.79943195 3.91700275]
 [1.00916657 0.37443192 3.91700275]
 [1.04772419 0.79943195 4.51587362]
 [1.04772419 0.37443192 4.51587362]
 [0.4488533  0.79943195 4.55443121]
 [0.4488533  0.37443192 4.55443121]]
[[-0.30118127  1.29943199  2.71391339]
 [-0.30118127  0.33693192  2.71391339]
 [ 1.81801818  1.29943199  4.03775412]
 [ 1.81801818  0.33693192  4.03775412]
 [ 1.25511398  1.29943199  4.93884913]
 [ 1.25511398  0.33693192  4.93884913]
 [-0.86408547  1.29943199  3.61500841]
 [-0.86408547  0.33693192  3.61500841]]
[[-0.4207549   1.31193199  2.59634269]
 [-0.4207549   0.47443193  2.59634269]
 [ 0.09

In [9]:
o3d.visualization.draw_geometries([pc,  *obbs])

In [12]:
pc

array([[-67.42678348, -61.81937082, 139.        ],
       [-67.1586085 , -61.81937082, 139.        ],
       [-66.89043352, -61.81937082, 139.        ],
       ...,
       [ 62.80922801,  26.81239447, 114.        ],
       [ 63.02917008,  26.81239447, 114.        ],
       [ 63.24911215,  26.81239447, 114.        ]])

In [129]:
for obb in obbs:
    print(obb)

OrientedBoundingBox: center: (1.20222, 0.305637, 4.80608), extent: 2.59025, 2.34004, 0.0301937)
OrientedBoundingBox: center: (-0.157601, 0.253691, 4.87155), extent: 2.24958, 1.09703, 0.0871501)
OrientedBoundingBox: center: (-1.63629, 0.197048, 4.98765), extent: 2.84801, 2.33859, 0.0301098)
OrientedBoundingBox: center: (-3.49622, 0.169737, 4.32253), extent: 2.83357, 2.17853, 0.0871013)
OrientedBoundingBox: center: (-0.0858239, 0.903439, 3.934), extent: 1.01072, 1.00911, 0.991217)
OrientedBoundingBox: center: (1.7488, 1.09112, 3.83969), extent: 0.977363, 0.923484, 0.762722)
OrientedBoundingBox: center: (-0.446544, 1.02256, 2.65252), extent: 1.39076, 0.766873, 0.561257)
OrientedBoundingBox: center: (-0.76212, 1.03241, 1.90839), extent: 1.30078, 0.723958, 0.636936)


In [130]:
np.asarray(pc.points).max(axis = 0), np.asarray(pc.points).min(axis = 0)

(array([ 6.96082212,  1.36095281, 17.22674124]),
 array([-8.22514367, -9.61407465,  5.87761238]))

In [1]:
import numpy as np
from mmdet3d.datasets import build_dataset
from mmcv import Config

# Load the configuration file
cfg = Config.fromfile('configs/sunrgbd/pointpillars_hv_secfpn_sunrgbd-3d-10class.py')

# Modify dataset paths if necessary
cfg.data.train.data_root = '/path/to/output_dir/'
cfg.data.train.ann_file = cfg.data.train.data_root + 'sunrgbd_infos_train.pkl'
cfg.data.val.data_root = '/path/to/output_dir/'
cfg.data.val.ann_file = cfg.data.val.data_root + 'sunrgbd_infos_val.pkl'
cfg.data.test.data_root = '/path/to/output_dir/'
cfg.data.test.ann_file = cfg.data.test.data_root + 'sunrgbd_infos_test.pkl'

# Build the dataset
dataset = build_dataset(cfg.data.train)

# Extract point clouds and bounding boxes
for i in range(len(dataset)):
    # Get data
    data = dataset[i]
    
    # Extract point clouds
    points = data['points'].data.numpy()
    
    # Extract bounding boxes
    gt_bboxes_3d = data['gt_bboxes_3d'].tensor.numpy()
    gt_labels_3d = data['gt_labels_3d'].numpy()
    
    # Print or process the point clouds and bounding boxes
    print(f'Point cloud {i}:', points)
    print(f'Bounding boxes {i}:', gt_bboxes_3d)
    print(f'Labels {i}:', gt_labels_3d)
    
    # Process the data as needed
    # For example, you can visualize or save the data here

AssertionError: MMCV==2.2.0 is used but incompatible. Please install mmcv>=2.0.0rc4, <2.2.0.

In [ ]:
estrinsics

array([[ 0.999677, -0.023916, -0.006473,  0.      ],
       [ 0.024777,  0.964971,  0.261185,  0.      ],
       [-0.      , -0.261261,  0.965251,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  1.      ]])